# Project 1

### Importing Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("2018_Statistical_Annex_Table_1.xlsx")

### Initial Interpretations

In [28]:
#we use the head function to get a quick sense of the structure of the data
df.head(20)

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,SDG 3,NaN,SDG 4.3,NaN,SDG 4.6,NaN,SDG 8.5,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Human Development Index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank,NaN
2,HDI rank,Country,Value,NaN,(years),NaN,(years),NaN,(years),NaN,(2011 PPP $),NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2017,NaN,2017,NaN,2017,a,2017,a,2017,NaN,2017,NaN,2016,NaN
4,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1,NaN
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2,NaN
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3,NaN
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4,NaN
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4,NaN


We see that the data is tabulated with some custom nested headers, the field names are spread across rows 3 and 4. In order to make the data more compatible the pandas dataframe structure we will remove the extraneous headings and have the data entries properly aligned. 

In [29]:
data = df[5:]

In [30]:
data.head(20)

,Unnamed: 0,Table 1. Human Development Index and its components,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1,NaN
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2,NaN
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3,NaN
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4,NaN
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4,NaN
10,6,Iceland,0.934879,NaN,82.912,NaN,19.3479,b,12.3638,c,45810.2,NaN,13,NaN,6,NaN
11,7,"Hong Kong, China (SAR)",0.932583,NaN,84.097,NaN,16.3257,NaN,12.0381,NaN,58419.7,NaN,2,NaN,8,NaN
12,7,Sweden,0.932805,NaN,82.625,NaN,17.6346,NaN,12.4261,NaN,47765.7,NaN,9,NaN,7,NaN
13,9,Singapore,0.932042,NaN,83.218,NaN,16.2,d,11.4747,NaN,82503.1,e,-6,NaN,8,NaN
14,10,Netherlands,0.930639,NaN,82.005,NaN,18.0448,NaN,12.19,NaN,47899.8,NaN,5,NaN,10,NaN


In [31]:
headers = df.iloc[1].values.tolist()

In [32]:
headers

[nan,
 nan,
 'Human Development Index (HDI) ',
 nan,
 'Life expectancy at birth',
 nan,
 'Expected years of schooling ',
 nan,
 'Mean years of schooling',
 nan,
 'Gross national income (GNI) per capita',
 nan,
 'GNI per capita rank minus HDI rank',
 nan,
 'HDI rank',
 nan]

In [33]:
headers[0] = "HDI Rank"
headers[1] = "Country"

In [34]:
data.columns = headers

In [37]:
data.head()

,HDI Rank,Country,Human Development Index (HDI),nan,Life expectancy at birth,nan,Expected years of schooling,nan,Mean years of schooling,nan,Gross national income (GNI) per capita,nan,GNI per capita rank minus HDI rank,nan,HDI rank,nan
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1,NaN
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2,NaN
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3,NaN
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4,NaN
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4,NaN


In [36]:
data.drop(nan)

""
5
6
7
8
9
10
11
12
13
14
